In [ ]:
MariaDB [(none)]> create database dbms7;
Query OK, 1 row affected (0.001 sec)

MariaDB [(none)]> use dbms7;
Database changed

MariaDB [dbms7]> create table produk (
    -> kode_produk VARCHAR(10) primary key,
    -> nama_produk VARCHAR(50),
    -> harga INT);
Query OK, 0 rows affected (0.015 sec)

MariaDB [dbms7]> create table transaksi (
    -> id_transaksi INT AUTO_INCREMENT primary key,
    -> kode_produk VARCHAR(10),
    -> jumlah INT,
    -> total_harga INT,
    -> FOREIGN KEY (kode_produk) REFERENCES produk(kode_produk));
Query OK, 0 rows affected (0.025 sec)

MariaDB [dbms7]> create table pelanggan (
    -> id_pelanggan INT AUTO_INCREMENT primary key,
    -> nama_pelanggan VARCHAR(50),
    -> alamat VARCHAR(100));
Query OK, 0 rows affected (0.016 sec)

MariaDB [dbms7]> insert into produk values ('P001', 'Laptop', 7000000), ('P002', 'Mouse', 150000);
Query OK, 2 rows affected (0.053 sec)
Records: 2  Duplicates: 0  Warnings: 0

MariaDB [dbms7]> insert into transaksi (kode_produk, jumlah, total_harga) values ('P001', 2, 14000000), ('P002', 5, 750000);
Query OK, 2 rows affected (0.007 sec)
Records: 2  Duplicates: 0  Warnings: 0

MariaDB [dbms7]> insert into pelanggan (nama_pelanggan, alamat) values ('Andi', 'Jl. Merdeka'), ('Budi', 'Jl.Sudirman');
Query OK, 2 rows affected (0.017 sec)
Records: 2  Duplicates: 0  Warnings: 0

MariaDB [dbms7]> CREATE INDEX idx_harga ON produk(harga);
Query OK, 0 rows affected (0.022 sec)
Records: 0  Duplicates: 0  Warnings: 0

MariaDB [dbms7]> EXPLAIN SELECT * FROM transaksi WHERE total_harga > 5000000;
+------+-------------+-----------+------+---------------+------+---------+------+------+-------------+
| id   | select_type | table     | type | possible_keys | key  | key_len | ref  | rows | Extra       |
+------+-------------+-----------+------+---------------+------+---------+------+------+-------------+
|    1 | SIMPLE      | transaksi | ALL  | NULL          | NULL | NULL    | NULL | 2    | Using where |
+------+-------------+-----------+------+---------------+------+---------+------+------+-------------+
1 row in set (0.010 sec)

MariaDB [dbms7]> SELECT * FROM produk WHERE harga > (SELECT AVG(harga) FROM produk);
+-------------+-------------+---------+
| kode_produk | nama_produk | harga   |
+-------------+-------------+---------+
| P001        | Laptop      | 7000000 |
+-------------+-------------+---------+
1 row in set (0.016 sec)

MariaDB [dbms7]> SELECT * FROM transaksi WHERE total_harga > 1000000;
+--------------+-------------+--------+-------------+
| id_transaksi | kode_produk | jumlah | total_harga |
+--------------+-------------+--------+-------------+
|            1 | P001        |      2 |    14000000 |
+--------------+-------------+--------+-------------+
1 row in set (0.000 sec)

MariaDB [dbms7]> ALTER TABLE transaksi ADD COLUMN id_pelanggan INT;
Query OK, 0 rows affected (0.021 sec)
Records: 0  Duplicates: 0  Warnings: 0

MariaDB [dbms7]> UPDATE transaksi SET id_pelanggan = 1 WHERE id_transaksi = 1;
Query OK, 1 row affected (0.017 sec)
Rows matched: 1  Changed: 1  Warnings: 0

MariaDB [dbms7]> UPDATE transaksi SET id_pelanggan = 2 WHERE id_transaksi = 2;
Query OK, 1 row affected (0.015 sec)
Rows matched: 1  Changed: 1  Warnings: 0

MariaDB [dbms7]> ALTER TABLE transaksi
    -> ADD CONSTRAINT fk_pelanggan
    -> FOREIGN KEY (id_pelanggan) REFERENCES pelanggan (id_pelanggan);
Query OK, 2 rows affected (0.102 sec)
Records: 2  Duplicates: 0  Warnings: 0

MariaDB [dbms7]> SELECT p.nama_pelanggan, SUM(t.total_harga) AS total_transaksi
    -> FROM pelanggan p
    -> JOIN transaksi t ON p.id_pelanggan = t.id_pelanggan
    -> GROUP BY p.nama_pelanggan;
+----------------+-----------------+
| nama_pelanggan | total_transaksi |
+----------------+-----------------+
| Andi           |        14000000 |
| Budi           |          750000 |
+----------------+-----------------+
2 rows in set (0.002 sec)

MariaDB [dbms7]> EXPLAIN
    -> SELECT p.nama_pelanggan, SUM(t.total_harga) AS total_transaksi
    -> FROM pelanggan p
    -> JOIN transaksi t ON p.id_pelanggan = t.id_pelanggan
    -> GROUP BY p.nama_pelanggan;
+------+-------------+-------+------+---------------+------+---------+------+------+-------------------------------------------------+
| id   | select_type | table | type | possible_keys | key  | key_len | ref  | rows | Extra                                           |
+------+-------------+-------+------+---------------+------+---------+------+------+-------------------------------------------------+
|    1 | SIMPLE      | p     | ALL  | PRIMARY       | NULL | NULL    | NULL | 2    | Using temporary; Using filesort                 |
|    1 | SIMPLE      | t     | ALL  | fk_pelanggan  | NULL | NULL    | NULL | 2    | Using where; Using join buffer (flat, BNL join) |
+------+-------------+-------+------+---------------+------+---------+------+------+-------------------------------------------------+
2 rows in set (0.001 sec)

MariaDB [dbms7]> CREATE UNIQUE INDEX idx_kode_produk ON produk(kode_produk);
Query OK, 0 rows affected (0.028 sec)
Records: 0  Duplicates: 0  Warnings: 0

MariaDB [dbms7]> EXPLAIN SELECT * FROM produk WHERE kode_produk= 'P001';
+------+-------------+--------+-------+-------------------------+---------+---------+-------+------+-------+
| id   | select_type | table  | type  | possible_keys           | key     | key_len | ref   | rows | Extra |
+------+-------------+--------+-------+-------------------------+---------+---------+-------+------+-------+
|    1 | SIMPLE      | produk | const | PRIMARY,idx_kode_produk | PRIMARY | 42      | const | 1    |       |
+------+-------------+--------+-------+-------------------------+---------+---------+-------+------+-------+
1 row in set (0.001 sec)